## **2.1 En C++**

In [ ]:
code= """
#include <mpi.h>
#include <iostream>

using namespace std;

void rellenar_vector(int vec[], int cant)
{
  for(int i=0;i<cant;i++)
  {
    vec[i]=rand()%5;
  }
}

void mostrar_error()
{
  int my_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  if(my_rank==0)
  {
    cout<<"----Error la cantidad de elementos debe ser multiplo de la cantidad de procesos, para poder asi aplicar scatter correctmente----"<<endl<<endl; 
  }
  MPI_Barrier(MPI_COMM_WORLD);
  MPI_Abort(MPI_COMM_WORLD,-1);
  exit(-1);
  

}

void verificar_valores(int *elementos, int nproc)
{
  if(*elementos<nproc)
  {
    *elementos=nproc;
  }
  if(*elementos%nproc!=0)
  {
    mostrar_error();
  }
}

void mostrar_vector(int vec[], int cant)
{
  cout<<"(";
  for(int i=0;i<cant;i++)
  {
    cout<<vec[i]<<",";
  }
  cout<<")"<<endl;
}

void crear_vectores(int **vec_a, int **vec_b,int **vec_c, int cant)
{
  *vec_a = (int *)malloc(cant*sizeof(int));
  *vec_b = (int *)malloc(cant*sizeof(int));
  *vec_c = (int *)malloc(cant*sizeof(int));

}

void realizar_suma(int vec_a[], int vec_b[],int vec_c[],int cant)
{
  for(int i=0; i<cant; i++)
  {
    vec_c[i]=vec_a[i]+vec_b[i];
    
  }

}

int main(int argc, char** argv) 
{
        int rank;
        int buf;
        int nproc;
        const int root=0;

        // Leo los parametros.
        if( argc != 2 )
        {
            cerr<< " Error en los parametros ."<<argc<<endl;
            exit( -1 );
        }

        srand(time(NULL));
        
        int elementos = atoi( argv[1] );
        int *vectorA, *vectorB, *vectorC, *vectorA_local,*vectorB_local,*vectorC_local;
        
        MPI_Init(&argc, &argv);
        MPI_Comm_size(MPI_COMM_WORLD, &nproc );
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

        verificar_valores(&elementos, nproc);
        
        vectorA_local=NULL;

        //Como vectorA_local y vector_localB lo van a tener que usar en scatter y en el bcast todos los procesos, 
        //en donde se van hacer las copias,incluido el master. Entonces se genera el new fuera del if

        vectorA_local=(int *)malloc((elementos/nproc)*sizeof(int));
        vectorB_local=(int *)malloc((elementos/nproc)*sizeof(int));      
        
        //como vectorlocalC tambien va necesaitar una copia en todos los procesos incluido el master,
        //entonces aca se genera el malloc
        vectorC_local=(int *)malloc((elementos/nproc)*sizeof(int));      
    
        if(rank==0)
        {
          crear_vectores(&vectorA,&vectorB,&vectorC,elementos);
          cout<<"vectorA: ";
          rellenar_vector(vectorA,elementos);
          mostrar_vector(vectorA,elementos);
          
          cout<<"vectorB: ";      
          rellenar_vector(vectorB,elementos);
          mostrar_vector(vectorB,elementos);

        }
        
        MPI_Scatter(vectorA, (elementos/nproc), MPI_INT, vectorA_local, (elementos/nproc), MPI_INT, 0, MPI_COMM_WORLD);
        MPI_Scatter(vectorB, (elementos/nproc), MPI_INT, vectorB_local, (elementos/nproc), MPI_INT, 0, MPI_COMM_WORLD);

        realizar_suma(vectorA_local, vectorB_local,vectorC_local,elementos/nproc);

        MPI_Gather(vectorC_local,(elementos/nproc),MPI_INT,vectorC,(elementos/nproc),MPI_INT,0,MPI_COMM_WORLD);
        
        if(rank==0)
        {
          cout<<"vectorC: ";
          mostrar_vector(vectorC,elementos);
        }
        MPI_Finalize();

        if(rank==0)
        {
          free(vectorA);
          free(vectorB);
          free(vectorC);
        }
        
        free(vectorA_local);
        free(vectorB_local);
        free(vectorC_local);
                
        
}
"""
text_file = open("addvector.c", "w")
text_file.write(code)
text_file.close()

In [ ]:
!mpic++ -o addvector addvector.c

In [ ]:
!mpirun --allow-run-as-root -oversubscribe  -np 2 addvector 6

## **2.2 En Python**